# TP 11 - Benchmark et complexité

Pour chaque exercice, rentrez votre réponse dans l'éditeur Python associé.
Enregistrez vos modifications, de préférence au format ipynb, lorsque vous aurez terminé.

## Exercice 1 - Génération de nombres aléatoires

Utiliser random et générer des sets de données aléatoires pour tester les algos suivants.

## Exercice 2 - Recherche linéaire versus recherche dichotomique

Générer un set de données.
Pour ce même set de données : linear search && bisection search => instrumenter & regarder

Instrumentez-le avec un wallclock et notez le temps pour dix, milles, cent milles, 1 million et 1 milliard d'éléments.

On doit pouvoir tracer à la mano un graphe qui montre la différence ouf entre les 2 (on introduira matplotlib dans un prochain TP)

## Exercice 3 - Tri

Que fait l'algo suivant :
<<<Bubble Sort>>>

Quel est sa complexité ?

Instrumentez-le avec un wallclock et notez le temps pour dix, milles, cent milles, 1 million et 1 milliard d'éléments.

Utiliser la fonction `sort` à la place. Testez également pour dix, milles, cent milles, 1 million et 1 milliard d'éléments. Qu'en déduisez-vous ?

## Exercice 4 - Générer tous les ensembles pouvant contenir les éléments d'un ensemble

Soit L = { 1, 2, 3 }

==> Générer en sortie la liste [ {}, { 1 }, { 2 }, { 3 }, { 1, 2 }, { 1, 3 }, { 2, 3 }, { 1, 2, 3 }]

Problème intrinsèquement exponentiel

==> Tester et mesurer pour 2, 5, 10, 20, 50, et 100 éléments.